# Table of Contents
| Section                      | Subsections                |
|-------------------------------|----------------------------|
|[Data Sources](#data-sources)|[Datasets](#datasets)|
|[Data Cleaning](#data-cleaning)| [Shapefiles](#shapefiles), [Rasters](#rasters)|
|[Data Exporting](#data-exporting)|--|

Contexto
Esto implica, que cualquier ahorro generado
por una mejora en la carga y el acarreo impactan directamente en un costo menor por tonelada de material
transportado [MODELO CONCEPTUAL DE SISTEMAS DE CARGA Y ACARREO DE MINERAL EN MINAS A CIELO ABIERTO, Salomón Liliana1
; Ortiz Alexis2 ]


https://es.linkedin.com/pulse/optimizaci%C3%B3n-y-simulaci%C3%B3n-de-minas-jorge-lozano

https://www.youtube.com/watch?v=KAK0RrkC2Qo

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import scipy.stats as stats


from ipywidgets import interact, Dropdown, widgets
from IPython.display import display

In [2]:
df = pd.read_csv('timeseries_haul_loading_data.csv')

## Analisis exploratorio

En la primera parte del analisis exploratorio se usara la libreria de autoreporting ydata_profiling. En la seccion de hallazgos se discute sobre la calidad de la data.

In [30]:
# profile = ProfileReport(df, title="Profiling Report")
profile

In [32]:
## Estadisticas basicas con desviacion estandar
df.describe()

,ton,n_shovel,truck_total_cycle,loader_total_cycle,distance_empty,distance_full
count,395680.000000,395680.000000,395680.000000,395680.000000,395680.000000,395680.000000
mean,318.007490,3.316556,2284.313481,305.073340,6161.241612,5782.712235
std,18.124753,0.558689,856.620246,116.657122,2569.465489,2355.404982
min,0.000000,3.000000,23.000000,0.000000,512.000000,504.000000
25%,305.666647,3.000000,1633.000000,217.000000,3931.000000,3754.000000
50%,317.758250,3.000000,2134.000000,303.000000,6009.000000,5412.000000
75%,330.125978,4.000000,2945.000000,390.000000,8059.000000,7798.000000
max,384.927167,15.000000,15812.000000,11692.000000,15783.000000,15691.000000


### Hallazgos
    
- **Datos categoricos**: 2 `truck`, `loader`<br>
    
    - `truck` tiene 47 valores únicos (47 camiones),
    - `loader` solo tiene 4 valores únicos (4 tipos de palas)
- **Datos faltantes**: No hay datos faltantes!

- La serie de tiempo abarca desde `2023-01-01` hasta `2024-05-24`.`

- **Datos numericos**: 6
    
   - El tonelaje (`ton`) promedio de carga es bastante consistente siendo de 318.00 t +/- 18.12. En promedio se requieren 3.32 paladas (`n_shovel`) para cargar un camion, la variabilidad aquí también es baja, sugiriendo una **estandarización en las operaciones de carga**. 
   - Los ciclos totales, tanto para camiones (`truck_total_cycle`) como para palas (`loader_total_cycle`), muestran una mayor variabilidad, lo que podría indicar **diferencias en la eficiencia o en las condiciones operativas a lo largo del tiempo**. 
   - Existe una fuerte correlacion entre `truck_total_cycle` y las distancias recorridas, tanto vacias (`distance_empty`) como llenas (`distance_full`), es decir, hay una relación directa entre el tiempo que un camión pasa en operación y la distancia que recorre, lo que podria tener implicaciones de eficiencia en **costos operativos** (más combustible y mantenimiento/desgaste), o la necesidad de mejorar la planificación de rutas.
   
   _Esto último no será considera en este análsis puesto que no se cuenta con información geográfica, pero vale la pena señalarlo_.



#### Identificacion de outliers
Inicialmente, se lleva a cabo una evaluación gráfica de la normalidad de las variables numéricas mediante el método Quantile-Quantile (Q-Q). Esto es necesario para evaluar si es posible la implementación directa de métodos estadísticos o de modelos de predicción que asuman una distribución normal de las variables, o si es requerido algún tipo de transformación y preprocesamiento de los datos.



In [5]:
@interact(variable=[var for var in df.columns if df[var].dtype in ['float64']])
def plot_qq(variable):
    plt.figure(figsize=(8, 6))
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.title(f'Gráfico Q-Q para {variable}')
    plt.grid(True)
    plt.show()


interactive(children=(Dropdown(description='variable', options=('ton', 'n_shovel', 'truck_total_cycle', 'loade…




### Posteriormente, se usara el rango intercuartílico para la identificacion de outliers ya que es menos sensible a los outliers extremos.

In [ ]:
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data < lower_bound) | (data > upper_bound)]

outliers_truck_cycle = detect_outliers_iqr(df['truck_total_cycle'])
outliers_distance_empty = detect_outliers_iqr(df['distance_empty'])

print("Outliers en Ciclo Total del Camión:", outliers_truck_cycle)
print("Outliers en Distancia Recorrida Vacía:", outliers_distance_empty)


In [ ]:
from scipy.stats import zscore

df['z_score_truck_cycle'] = zscore(df['truck_total_cycle'])
df['z_score_distance_empty'] = zscore(df['distance_empty'])

outliers_truck_cycle_z = df[df['z_score_truck_cycle'].abs() > 3]
outliers_distance_empty_z = df[df['z_score_distance_empty'].abs() > 3]

print("Outliers en Ciclo Total del Camión (Z-score):", outliers_truck_cycle_z)
print("Outliers en Distancia Recorrida Vacía (Z-score):", outliers_distance_empty_z)


In [35]:
outliers_truck_cycle

514        5265.0
1013      15033.0
3007       7340.0
3118       6253.0
3344       9768.0
           ...   
391771     5594.0
393165     6098.0
393930     4923.0
393993     6152.0
394796     5141.0
Name: truck_total_cycle, Length: 1154, dtype: float64

# Soluciones
### Clustering para Segmentación de Equipos Basado en Rendimiento:

Descripción: Aplica técnicas de clustering (como K-means o Clustering Jerárquico) para segmentar los equipos en grupos basados en su rendimiento y características operativas. Esto podría revelar patrones como cuales combinaciones de palas y camiones trabajan mejor juntos.
Simplicidad y Eficiencia: K-means es un algoritmo relativamente simple y eficaz para identificar grupos. La eficiencia se puede asegurar eligiendo un número adecuado de clusters a través del método del codo o análisis de silueta.
Uso del Modelo: Los clusters pueden ayudar a identificar grupos de alto y bajo rendimiento, y explorar las características operativas que diferencian estos grupos, lo que puede informar estrategias de asignación de recursos.

### Dashboards Interactivos
#### Dashboard de Monitoreo de Rendimiento Diario:

KPIs Relevantes:
Tonelaje total movido por día por equipo.
Ciclos totales de carga por camión y pala.
Eficiencia de carga (tonelaje por palada).
Tiempos de ciclo (ciclo total del camión y de la pala).
Características:
Filtros interactivos por fecha, tipo de equipo, y código de equipo.
Gráficos de tendencias de rendimiento a lo largo del tiempo.
Tablas de clasificación que se actualizan dinámicamente para mostrar los equipos de mejor rendimiento.

#### Dashboard de Análisis de Factores Críticos:

KPIs Relevantes:
Correlaciones entre variables operativas y tonelaje movido.
Importancia de las variables (basada en modelos predictivos).
Distribuciones de las variables clave por segmentos de alto y bajo rendimiento.
Características:
Visualizaciones de calor y gráficos de dispersión para explorar relaciones entre variables.
Histogramas y gráficos de cajas para analizar la distribución de variables críticas.
Mapas de calor para mostrar la correlación entre diferentes variables y el rendimiento.